In [1]:
#import
import math
from scipy.integrate import solve_ivp
import numpy as np

from collections import deque
import json

#overall arguments
#for simulating by the scipy ODE solver
maxT = 2.0 #max time for evolution inside a rectangle
rtol=1e-7
M=2
#solve_ivp(fun, t_span, y0, method, t_eval, dense_output, events, vectorized, args, **options)
#method in ['RK23', 'RK45', 'Radau', 'BDF', 'LSODA']
Method='LSODA' #method argument is not necessary
t_eval_points_count = 100 #400
#QDA arg
K=3

In [2]:
#INPUT Van der Pol Oscillator
#input to classic QDA

#ODE for 2 variables
def vanderpol_f(t, y):
    x0,x1 = y
    return np.array([ x1, 
                      2 * ( 1 - x0 * x0 ) * x1 - x0 ])

def vanderpol_f_auton(y):
    return vanderpol_f(0,y)

#minus van der pol
def m_vanderpol_f(t, y):
    x0,x1 = y
    return np.array([ -x1, 
                      (-2) * ( 1 - x0 * x0 ) * x1 + x0 ])

def m_vanderpol_f_auton(y):
    return m_vanderpol_f(0,y)

#tresholds
x0min = -8
x0max = 8
x1min = -8
x1max = 8

pieces_count = 100

stepx0 = ( x0max - x0min ) / pieces_count
stepx1 = ( x1max - x1min ) / pieces_count

tresholdsX0 = np.arange ( x0min, x0max+(0.3*stepx0), stepx0 )
tresholdsX1 = np.arange ( x1min, x1max+(0.3*stepx1), stepx1 )

tresholds = [ tresholdsX0, tresholdsX1 ]

In [3]:
#approximation by multi-affine using the derivative at vertices
#supposing a rectangle exists this finds it
def find_rectangle( x ):
    r = [0,0]
    for i in [0,1]:
        for j in range(0, pieces_count):
            if (tresholds[i][j] < x[i]):
                r[i] = j        
    return r

def approx_func_in_r_auton( x, r, func_auton ):
    a = tresholds[0][ r[0] ]
    b = tresholds[0][ r[0]+1 ]
    c = tresholds[1][ r[1] ]
    d = tresholds[1][ r[1]+1 ]
    
    v00 = [ a, c ]
    v01 = [ a, d ]
    v10 = [ b, c ]
    v11 = [ b, d ]
    
    f00 = func_auton( v00 )
    f01 = func_auton( v01 )
    f10 = func_auton( v10 )
    f11 = func_auton( v11 )
    
    coef0 = ( x[0] - v00[0] ) / ( v10[0] - v00[0] )
    coef1 = ( x[1] - v00[1] ) / ( v01[1] - v00[1] )
    
    f0c = coef0*f00 + (1-coef0)*f10
    f1c = coef0*f01 + (1-coef0)*f11
    fx = coef1*f0c + (1-coef1)*f1c
    
    return fx

def approx_func_anywhere_auton( x, func_auton ):
    r = find_rectangle( x )
    return approx_func_in_r_auton( x, r, func_auton )

def approx_func_anywhere( x, t, func_auton ):
    return approx_func_anywhere_auton( x, func_auton )

In [4]:
#is the rectangle inside our system of tresholds?
def exists_rectangle( r ):
    exists = True
    for i in [0,1]:
        if r[i] < 0 :
            exists = False
        elif r[i] > (pieces_count-1): #cannot start on the last treshold or higher
            exists = False
    return exists

def is_point_inside_rectangle( x, r ):
    inside = True
    for i in [0,1]:
        minim = (tresholds[i])[ r[i] ]
        maxim = (tresholds[i])[ r[i]+1 ]
        if x[i] < minim: inside = False
        if x[i] > maxim: inside = False
    return inside

#outside rectangle, below lower facet
def is_point_below_facet( x, r, direction ):
    minim_dir = (tresholds[ direction ])[ r[ direction ] ]
    return ( x[ direction ] < minim_dir)

#outside rectangle, above upper facet
def is_point_above_facet( x, r, direction ):
    maxim_dir = (tresholds[ direction ])[ r[ direction ]+1 ]
    return ( x[ direction ] > maxim_dir )

#sharp <0 with outside normal vector to facet
def RA_inside_condition( x, direction, orientation, derivative_func ):
    i_der_at_x = (derivative_func( x ))[ direction ]
    #print("deriv at x="+str(x)+" is="+str(i_der_at_x))
    if orientation == 0 : # |-> |
        #print("RA condition="+str( i_der_at_x > 0 ))
        return ( i_der_at_x > 0 )
    else: # |<-|
        #print("RA condition="+str( i_der_at_x < 0 ))
        return ( i_der_at_x < 0 )
    
#sharp >0 with outside normal vector to facet
def RA_outside_condition( x, direction, orientation, derivative_func ):
    i_der_at_x = (derivative_func( x ))[ direction ]
    if orientation == 0 : # <-| |
        return ( i_der_at_x < 0 )
    else: # | |->
        return ( i_der_at_x > 0 )

#check if 0 vector can be obtained as a convex combination of
#derivative values at corners of rectangle r
def RA_selfloop_condition( r, derivative_func ):
    if not exists_rectangle: 
        return False
    else:
        existsP = [ False, False ]
        existsN = [ False, False ]
        a = [ tresholds[0][ r[0]     ], tresholds[1][ r[1]     ] ]
        b = [ tresholds[0][ r[0] + 1 ], tresholds[1][ r[1]     ] ]
        c = [ tresholds[0][ r[0]     ], tresholds[1][ r[1] + 1 ] ]
        d = [ tresholds[0][ r[0] + 1 ], tresholds[1][ r[1] + 1 ] ]
        for v in [ a, b, c, d ]:
            fv = derivative_func( v )
            if( fv[0] > 0 ): existsP[0] = True
            if( fv[0] < 0 ): existsN[0] = True
            if( fv[1] > 0 ): existsP[1] = True
            if( fv[1] < 0 ): existsN[1] = True
        return existsP[0] and existsN[0] and existsP[1] and existsN[1]

def which_facets_outside( x, r ): #which is my exit facet of r
    result = [ [], [] ] #exit facets on dim 0, 1
    for fdir in [0,1]:
        if is_point_below_facet( x, r, fdir ): #(outside) below lower facet
            result[ fdir ] = [0]
        if is_point_above_facet( x, r, fdir ): #(outside) above upper facet
            result[ fdir ] = [1]
    return result

def exit_point_from_segment( xin, xout, r, outside_facets ):
    new_outside_facets = outside_facets
    xexit_result = xout #default value
    for i in [0,1]:
        if len( outside_facets[i] ) > 0: #i.e. len( outside_facets[i] )==1
            #find exit point on hyperplane
            xexit=[0.0,0.0] #default
            xexit[ i ] = (tresholds[i])[ r[i] + outside_facets[i][0]] #constant on facet
            coef = ( xexit[ i ] - xin[ i ] )/( xout[ i ] - xin[ i ] ) 
            xexit[ 1 - i ] = ( xout[ 1-i ] - xin[ 1-i ] ) * coef + xin[ 1-i ]
            #if in the facet (in r), let outside facets be otherwise delete this facet
            if is_point_inside_rectangle( xexit, r ):
                xexit_result = xexit
            else:
                new_outside_facets[i] = []
    return [ xexit_result, new_outside_facets ] #the exit point inside r

def real_vertices( r, fdir, fori ):
    #real coordinates of (2) vertices of a (1dim) facet
    v1 = [0,0]
    v2 = [0,0]
    v1[ fdir ] = (tresholds[ fdir ])[ r[ fdir ] + fori ]
    v2[ fdir ] = (tresholds[ fdir ])[ r[ fdir ] + fori ]
    v1[ 1-fdir ] = (tresholds[ 1-fdir ])[ r[ 1-fdir ] ]
    v2[ 1-fdir ] = (tresholds[ 1-fdir ])[ r[ 1-fdir ] + 1 ]
    return [v1,v2]


In [5]:
def approx_tiles( x, k, r, fdir ): #x is a 0dim real number
    fmin = (tresholds[ 1-fdir ])[ r[ 1-fdir ] ]
    fmax = (tresholds[ 1-fdir ])[ r[ 1-fdir ] + 1 ]
    xnorm = (x-fmin)/(fmax-fmin)
    tilemin = math.floor( k*xnorm )
    tilemax = math.ceil( k*xnorm )
    tilemin = max( tilemin, 0)
    tilemax = min( tilemax, k)
    return [ tilemin, tilemax ]

def nonempty_intersection( tiles1, tiles2 ):
    return ( max(tiles1[0],tiles2[0]) <=  min(tiles1[1],tiles2[1]) )
    
def get_intersection( tiles1, tiles2 ):
    if nonempty_intersection( tiles1, tiles2 ):
        return [ max( tiles1[0], tiles2[0] ), min( tiles1[1], tiles2[1] ) ]

def unite_tiles( tiles1, tiles2 ): #overapproximate the union by one tileset
    return [ min( tiles1[0], tiles2[0] ), max( tiles1[1], tiles2[1]) ]

def unite_exactly_tiles( tiles1, tiles2 ): #list of tilesets
    if nonempty_intersection( tiles1, tiles2 ):
        return [ unite_tiles(tiles1,tiles2) ]
    else:
        return [ tiles1, tiles2 ]
    
def is_subset_of( subs, supers ):
    if len(subs) == 0:
        return True
    elif len(supers) == 0:
        return False
    else:
        return (subs[0] >= supers[0]) and (subs[1] <= supers[1])
    

def non_null_tiles( tiles_dict, k ): #enlarge tiles to nonzero (n-1)measure (no isolated points)
    new_dict = tiles_dict
    for key in tiles_dict:
        tiles = tiles_dict[ key ]
        if len( tiles ) == 2 :
            if( tiles[0] == tiles[1] ):
                if( tiles[0] == 0 ):
                    new_dict[ key ] = [ 0, 1 ]
                elif( tiles[0] == k ):
                    new_dict[ key ] = [ k - 1, k ]
                else:
                    q = tiles[0]
                    new_dict[ key ] = [ q - 1, q + 1 ]
    return new_dict

def RA_approved_exit( exit_sets_dict, r, k, ODEfunc_auton ):
    #for exit facets (key) that correspond to a nonempty portion of the entry set (tiles)
    #identify the exit facet from (key)
    #in case key != sefloop, find vertices of exit facet and check RA exit condition
    #    condition not satisfied => exit not possible from this facet, not included in exit sets anymore,
    #    successors will not be calculated from no-exit facets
    #    condition satisfied => exit facet stays in the list
    #in case key == sefloop, we may check RA approved selfloop from the derivatives at vertices of r
    #    condition not satisfied => result of small maxT, issue warning, not include in successors
    #    condition satisfied => selfloop stays
    new_dict = exit_sets_dict
    for key in exit_sets_dict:
        tiles = exit_sets_dict[ key ]
        if( len(tiles) == 2 ): #check nonempty entry sets
            #selfloop
            if key == 'inside':
                if not RA_selfloop_condition( r, ODEfunc_auton ):
                    #print( "Rectangle "+str(r)+" is not a RA approved selfloop.")
                    new_dict[ "inside" ] = []
                else:
                    #print( "Rectangle "+str(r)+" is SELFLOOP." )
                    print( str(r) )
            #regular exit facet
            else:
                if key[0] == '0': fdir = 0
                else: fdir = 1
                if key[1] == '0': fori = 0 
                else: fori = 1
                v1,v2 = real_vertices( r, fdir, fori )
                if not RA_outside_condition( v1, fdir, fori, ODEfunc_auton ) and not RA_outside_condition( v2, fdir, fori, ODEfunc_auton ):
                    new_dict[ key ] = []
    return new_dict

In [6]:
#get successors routine
#input =  k ... argument for entry set approximation by (a rectangle of) tiles
#         r - rectangle ... list of int coordinates of starting tresholds (<pieces_count), 
#         e - entry set ... [entry var, entry facet], 
#                           dim-1 coordinates of tiles rectangle boundary
#         m - number of simulations per tile
#output = list of successors with entry sets (and labels)

#forward sims only for now

def get_successors( k, r, e_facet, e, ODEfunc, ODEfunc_auton ):
    e_dir,e_or = e_facet
    successors_dict = { "00":0, "01":0, "10":0, "11":0, "inside":0 } #my exit facets
    exit_sets_dict = { "00":[], "01":[], "10":[], "11":[], "inside":[] } #my exit facets : my exit tiles
    points_to_simulate = []
        
    if len(e) == 2:
        #coordinates that are changing on the facet
        rmin_facet = (tresholds[ 1-e_dir ])[ r[1-e_dir] ]
        rmax_facet = (tresholds[ 1-e_dir ])[ r[1-e_dir] + 1 ]
        Kstep = (rmax_facet - rmin_facet)/(K*1.0) #float
        
        #the coordinate that is constant on the facet
        e_const = (tresholds[ e_dir ])[ r[e_dir] + e_or ]
        #the entry set changing coords between
        emin = rmin_facet + e[0]*Kstep
        emax = rmin_facet + e[1]*Kstep
        
        if e[0] < e[1]: #nontrivial 1d entry set
            Mstep = Kstep / (M*1.0) #float
            tiles = np.arange( emin, emax, Kstep )
            
    
            #for all entry tiles simulate and count
            #ti is a beginning of tile
            for ti in tiles:
                for i in range( 0, M ):
                    if e_dir == 0 :
                        x00 = e_const 
                        x01 = ti + i*Mstep
                    else:
                        x00 = ti + i*Mstep
                        x01 = e_const
                    points_to_simulate += [ [x00,x01] ]
        else: #1 point entry set (0dim), one trajectory will be simulated
            if e_dir == 0 :
                x00 = e_const 
                x01 = emin
            else:
                x00 = emin
                x01 = e_const
                points_to_simulate += [ [x00,x01] ]
    else:
        
        rmin0 = (tresholds[ 0 ])[ r[0] ]
        rmax0 = (tresholds[ 0 ])[ r[0] + 1 ]
        rmin1 = (tresholds[ 1 ])[ r[1] ]
        rmax1 = (tresholds[ 1 ])[ r[1] + 1 ]
        Kstep0 = (rmax0 - rmin0)/(K*1.0) #float
        Kstep1 = (rmax1 - rmin1)/(K*1.0) #float
            
        Mstep0 = Kstep0 / (M*1.0) #float
        Mstep1 = Kstep1 / (M*1.0) #float
        tiles0 = np.arange( rmin0, rmax0, Kstep0 )
        tiles1 = np.arange( rmin1, rmax1, Kstep1 )
    
        #for all entry tiles simulate and count
        #ti is a beginning of tile
        for ti0 in tiles0:
            for ti1 in tiles1:
                for i in range( 0, math.ceil( math.sqrt(M) ) ):
                    for l in range( 0, math.ceil( math.sqrt(M) ) ):
                        x00 = ti0 + i*Mstep0
                        x01 = ti1 + l*Mstep1
                        points_to_simulate += [ [x00,x01] ]
    
    RA_unsat_count = 0
    RA_sat_count = 0
    for x0 in points_to_simulate:
            x00 = x0[0]
            x01 = x0[1]
            #check the RA condition at x00,x01
            if RA_inside_condition( [x00,x01], e_dir, e_or, ODEfunc_auton ):
                RA_sat_count += 1
                sol = solve_ivp( ODEfunc, [0.0,maxT], [ x00, x01 ], Method, 
                                 t_eval=np.linspace( 0, maxT, t_eval_points_count ),
                                 rtol=rtol )
                #print(sol)
                #where does this trajectory go?
                j=1
                x = [ sol.y[0][j], sol.y[1][j] ]
                #print("x0="+str([x00,x01]))
                #print("x1="+str(x))
                #print("x2="+str( [ sol.y[0][2], sol.y[1][2] ] ))
                
                while (j < (t_eval_points_count-1)) and is_point_inside_rectangle( x, r ):
                    j += 1
                    x = [ sol.y[0][j], sol.y[1][j] ]
                    #print("j="+str(j))
                if is_point_inside_rectangle( x, r ):
                    successors_dict["inside"] = successors_dict["inside"] + 1
                else:
                    #where was the exit/entry point?
                    outside_facets = which_facets_outside( x, r ) #list of my exit facets
                    xin = [ sol.y[0][j-1], sol.y[1][j-1] ] #last inside
                    #compute the exit point and real
                    xexit,outside_facets = exit_point_from_segment( xin, x, r, outside_facets )
                    for fdir in [0,1]:
                        if 0 in outside_facets[ fdir ]: #my exit facets
                            successors_dict[ str(fdir)+"0" ] += 1
                            #update exit sets:
                            newtiles = approx_tiles( xexit[1-fdir], k, r, fdir )
                            if exit_sets_dict[ str(fdir)+"0" ] == []:
                                exit_sets_dict[ str(fdir)+"0" ] = newtiles
                            else:
                                exit_sets_dict[ str(fdir)+"0" ] = unite_tiles( newtiles, exit_sets_dict[ str(fdir)+"0" ] )
                        if 1 in outside_facets[ fdir ]:
                            successors_dict[ str(fdir)+"1" ] += 1
                            newtiles = approx_tiles( xexit[1-fdir], k, r, fdir )
                            if exit_sets_dict[ str(fdir)+"1" ] == []:
                                exit_sets_dict[ str(fdir)+"1" ] = newtiles
                            else:
                                exit_sets_dict[ str(fdir)+"1" ] = unite_tiles( newtiles, exit_sets_dict[ str(fdir)+"1" ] )
            else:
                RA_unsat_count += 1

    #print( RA_sat_count )
    #print( RA_unsat_count )
    #print( successors_dict )
    #print( exit_sets_dict )
    # do not enlarge the entry sets:
    # #exit_sets_dict = non_null_tiles( exit_sets_dict, k )
    #print( exit_sets_dict )
    exit_sets_dict = RA_approved_exit( exit_sets_dict, r, k, ODEfunc_auton )
    #print( exit_sets_dict )
    #result = [ successors_dict, exit_sets_dict ]
    successors_list = exitsets_dict_into_states_list( r, exit_sets_dict )
    #if successors_dict["inside"]>0: successors_list.append( [r,"inside",[]] )
    result = [ exit_sets_dict, successors_list ]#successors existing rectangles
    return result

#meaningful conversion applicable only to regular facets not selfloops
def key_into_facet( key ):
    if( key == 'inside' ): return 'inside'
    facet = [0,0]
    if( key[0] == '0' ):
        facet[0] = 0
    else:
        facet[0]= 1
    if( key[1] == '0' ):
        facet[1] = 0
    else:
        facet[1] = 1
    return facet

#meaningful conversion only for regular facets
def hash_f( facet ):
    if( facet == 'inside' ): return 'inside'
    return str(facet[0])+str(facet[1])

def hash_r( r ):
    hashr = str(r[0])+"|"+str(r[1])
    return hashr

def r_hash( h ):
    r = [0,0] #default value
    hparts = h.split('|')
    r[0] = int( hparts[0] )
    r[1] = int( hparts[1] )
    return r

#applicable only to regular facets not selfloops
def key_into_facet_in_successor( key ):
    facet = [0,0]
    if( key[0] == '0' ):
        facet[0] = 0
    else:
        facet[0]= 1
    if( key[1] == '0' ):
        facet[1] = 1 #the opposite than in the predecessor
    else:
        facet[1] = 0 #ditto
    return facet

#applicable only to regular facets not selfloops
def key_into_rectangle( key, r ): #key=facet of r, into successor rectangle of r
    rectangle = []
    rectangle.append(r[0])
    rectangle.append(r[1])
    facet = key_into_facet( key ) #dir,ori
    if( facet[1] == '0' ):
        rectangle[ facet[0] ] = rectangle[ facet[0] ] - 1
    else:
        rectangle[ facet[0] ] = rectangle[ facet[0] ] + 1
    return rectangle

#regular states and selfloop
#input = rs exit sets 
#output = list of successors <succ, succs entry set>
def exitsets_dict_into_states_list( r, exit_sets_dict ):
    result = []
    for key in exit_sets_dict:
        if len(exit_sets_dict[ key ]) > 0: #exitsets with nonempty portion of entry set
            if( key == 'inside' ): #set facet as string 'inside'
                result.append( [ r, 'inside', exit_sets_dict[ key ] ] )
            else:
                facet = key_into_facet_in_successor( key )
                rectangle = key_into_rectangle( key, r )#compute the right neighbouring rectangle
                if( exists_rectangle( rectangle )):
                    result.append( [ rectangle, facet, exit_sets_dict[key] ] )
    return result

In [7]:
#use the get successors routine K is defined in the first cell

#succs_dictionary = get_successors( K, [60,50], [0,0], [], vanderpol_f, vanderpol_f_auton )
#succs_dictionary = get_successors( K, [62,62], [1,1], [0,K], vanderpol_f, vanderpol_f_auton )
#succs_dictionary = get_successors( K, [43,37], [0,1], [0,K], vanderpol_f, vanderpol_f_auton )
#succs_dictionary = get_successors( K, [62,49], [0,1], [0,K], vanderpol_f, vanderpol_f_auton )
#succs_dictionary = get_successors( K, [43,37], [0,0], [0,K], vanderpol_f, vanderpol_f_auton )
##succs_dictionary = get_successors( K, [43,37], [1,0], [0,K], vanderpol_f, vanderpol_f_auton )
succs_dictionary = get_successors( K, [43,25], [0,1], [0,K], vanderpol_f, vanderpol_f_auton )
#?moc nejde spocitat, asi jiny init...succs_dictionary = get_successors( K, [43,22], [1,1], [0,K], m_vanderpol_f, m_vanderpol_f_auton )

print(succs_dictionary)

[{'00': [1, 3], '01': [], '10': [], '11': [0, 2], 'inside': []}, [[[44, 25], [0, 1], [1, 3]], [[43, 26], [1, 0], [0, 2]]]]


In [8]:
def update_explored_for_state( explored, s, facet, entry ): #i.e. add entry
    if hash_r(s) in explored:
        new_explored_s = explored[ hash_r(s) ]
        
        if hash_f(facet) in explored[ hash_r(s) ]:
            #update list of explored entry sets to s through facet
            #by union with entry
            #version 0.1 = just overapproximate the union by one tileset
            #result will be list with one element (future possibly a list of more disjunct elements)
            new_explored_s[ hash_f(facet) ] = [ unite_tiles( explored[ hash_r(s) ][ hash_f(facet) ][0], entry ) ]
        else:
            #create new list and list the explored entry set
            new_explored_s[ hash_f(facet) ] = [ entry ]
    else:
        new_explored_s = { hash_f(facet) : [ entry ] }
    return new_explored_s
        
#applicable to regular facets
def is_already_explored( explored, s, facet, entry ):
    if hash_r(s) in explored:
        if hash_f(facet) in explored[ hash_r(s) ]:
            return is_subset_of( entry, explored[ hash_r(s) ][ hash_f(facet) ][0] )
        else:
            return False
    else:
        return False
    
#visited reflects the entry-exit relations
#version 0.1= facet to facet
#future - more nuanced entry and exit sets (i.e. more possibilities)
def update_visited_for_state( visited, s, facet, entry, exit_sets ):
    if hash_r(s) in visited:
        new_visited_s = visited[ hash_r(s) ] #dictionary to update
        if hash_f(facet) in new_visited_s: #some exit sets for this entry facet listed
            for facet_key in exit_sets:
                if len(exit_sets[ facet_key ]) > 0: #exit set != []
                    if facet_key not in new_visited_s[ hash_f(facet) ]:
                        new_visited_s[ hash_f(facet) ].append( facet_key )
        else: #exit sets for this entry facet not listed yet
            new_visited_s[ hash_f(facet) ] = []
            for facet_key in exit_sets:
                if len(exit_sets[ facet_key ]) > 0: #exit set != []
                    new_visited_s[ hash_f(facet) ].append( facet_key )
    else:#rectangle not visited yet, make new dictionary in visited
        exit_sets_list = []
        for facet_key in exit_sets:
            if len(exit_sets[ facet_key ]) > 0: #entry set going to exit facet != []
                exit_sets_list.append( facet_key )
        new_visited_s = { hash_f(facet) : exit_sets_list }
    return new_visited_s


#find all reachable states from one state
#e_facet: [dir,or]
#e: [0,K] or [i,j] or []=whole rectangle (then e_facet value can be whatever)
def find_all_reachable_from( k, r, e_facet, e, ODEfunc, ODEfunc_auton, ident ):
    visited = {} #2d dict of rectangle : { facet : [ list of exit facets ] }
    explored = {} # { rectangle : [ entry, entrytiles ] }
    selfloops = [] # list of rectangles with (RA approved) selfloops
    queue = deque([])
    queue.append( [r, e_facet, e] )
    state_counter = 0
    while len( queue )>0 :
        #print( "len Q="+str(len(queue)) )
        state_counter += 1
        if (state_counter % 1000) == 0:
            write_out( visited, explored, selfloops, str(state_counter)+"Q"+str(len(queue))+ident )
        s,facet,entry = queue.pop()
        #print( s )
        #print( explored )
        #print( visited )
        #find successors where rectangles exist
        #a lot could have happened since this s,f,e was added to queue...
        #...check if a similar state has not been explored yet and then
        #possibly compute its successors
        if not is_already_explored( explored, s, facet, entry ):
            exit_sets,succs_list = get_successors( k, s, facet, entry, ODEfunc, ODEfunc_auton )
            #print( succs_list )
            explored[ hash_r(s) ] = update_explored_for_state( explored, s, facet, entry )
            #mark state as visited
            visited[ hash_r(s) ] = update_visited_for_state( visited, s, facet, entry, exit_sets )
            #enqueu successors
            for succ in succs_list:
                s_r,s_f,s_e = succ
                #selfloop successors are stored but not included in the queue further:
                if s_f == 'inside':
                    selfloops.append( hash_r(s_r) )
                elif not is_already_explored( explored,s_r,s_f,s_r ):
                    queue.append( succ )
    write_out( visited, explored, selfloops, "Konec"+ident )
    return visited #for the intersection we need only transient (entry-exit pairs)

def write_out( dictionar, dictionar2, listik, i ):
    #write content to the output file
    fo = open("./outVDPforward/output"+str(i)+".txt", "w")
    #fo.write("Visited:\n")
    fo.write( str(dictionar) )
    #fo.write( "\n\nExplored:\n" )
    #fo.write( str(dictionar2) )
    #fo.write( "\n\nSelfloops:\n")
    #fo.write( str(listik))
    fo.close()
   

In [9]:
#Find forward reachable sets for rectangles (entry [] the whole rectangle) in init
#write out the results - name them with identifiers "_x_y"
initial_rectangles = []
identifiers = []
for x in [-3, -2, -1, 0, 1, 2, 3]:
    for y in [-3, -2, -1, 0, 1, 2, 3]:
        initial_rectangles.append( find_rectangle( [ x, y ] ) )
        identifiers.append( "_" + str( x ) + "_" + str( y ) )

for r,i in zip( initial_rectangles, identifiers ):
    visited_vdp = find_all_reachable_from(  K, r, [0,0], [], vanderpol_f, vanderpol_f_auton, i )
    #print( visited_vdp )


{'31|31': {'00': []}}
{'31|37': {'00': []}}
{'31|43': {'00': []}}
{'31|49': {'00': []}}
{'31|56': {'00': ['10']}, '31|57': {'11': ['10']}, '31|58': {'11': ['10']}, '31|59': {'11': ['10']}, '31|60': {'11': ['10']}, '31|61': {'11': ['10']}, '31|62': {'11': ['10']}, '31|63': {'11': ['10']}, '31|64': {'11': ['10']}, '31|65': {'11': ['10']}, '31|66': {'11': ['10']}, '31|67': {'11': ['10']}, '31|68': {'11': ['10']}, '31|69': {'11': ['10']}, '31|70': {'11': ['10']}, '31|71': {'11': ['10']}, '31|72': {'11': ['10']}, '31|73': {'11': ['10']}, '31|74': {'11': ['10']}, '31|75': {'11': ['10']}, '31|76': {'11': ['10']}, '31|77': {'11': ['10']}, '31|78': {'11': ['10']}, '31|79': {'11': ['10']}, '31|80': {'11': ['10']}, '31|81': {'11': ['10']}, '31|82': {'11': ['10']}, '31|83': {'11': ['10']}, '31|84': {'11': ['10']}, '31|85': {'11': ['10']}, '31|86': {'11': ['10']}, '31|87': {'11': ['10']}, '31|88': {'11': ['10']}, '31|89': {'11': ['10']}, '31|90': {'11': ['10']}, '31|91': {'11': ['10']}, '31|92': {'

{'37|62': {'00': ['01', '10']}, '37|63': {'11': ['01', '10']}, '37|64': {'11': ['01', '10']}, '37|65': {'11': ['01', '10']}, '37|66': {'11': ['01', '10']}, '37|67': {'11': ['01', '10']}, '37|68': {'11': ['01', '10']}, '37|69': {'11': ['01', '10']}, '37|70': {'11': ['01', '10']}, '37|71': {'11': ['01', '10']}, '37|72': {'11': ['01', '10']}, '37|73': {'11': ['01', '10']}, '37|74': {'11': ['01', '10']}, '37|75': {'11': ['01', '10']}, '37|76': {'11': ['01', '10']}, '37|77': {'11': ['01', '10']}, '37|78': {'11': ['01', '10']}, '37|79': {'11': ['01', '10']}, '37|80': {'11': ['01', '10']}, '37|81': {'11': ['01', '10']}, '37|82': {'11': ['01', '10']}, '37|83': {'11': ['01', '10']}, '37|84': {'11': ['01', '10']}, '37|85': {'11': ['01', '10']}, '37|86': {'11': ['01', '10']}, '37|87': {'11': ['01', '10']}, '37|88': {'11': ['01', '10']}, '37|89': {'11': ['01', '10']}, '37|90': {'11': ['01', '10']}, '37|91': {'11': ['01', '10']}, '37|92': {'11': ['01', '10']}, '37|93': {'11': ['01', '10']}, '37|94'

{'37|68': {'00': ['01', '10']}, '37|69': {'11': ['01', '10']}, '37|70': {'11': ['01', '10']}, '37|71': {'11': ['01', '10']}, '37|72': {'11': ['01', '10']}, '37|73': {'11': ['01', '10']}, '37|74': {'11': ['01', '10']}, '37|75': {'11': ['01', '10']}, '37|76': {'11': ['01', '10']}, '37|77': {'11': ['01', '10']}, '37|78': {'11': ['01', '10']}, '37|79': {'11': ['01', '10']}, '37|80': {'11': ['01', '10']}, '37|81': {'11': ['01', '10']}, '37|82': {'11': ['01', '10']}, '37|83': {'11': ['01', '10']}, '37|84': {'11': ['01', '10']}, '37|85': {'11': ['01', '10']}, '37|86': {'11': ['01', '10']}, '37|87': {'11': ['01', '10']}, '37|88': {'11': ['01', '10']}, '37|89': {'11': ['01', '10']}, '37|90': {'11': ['01', '10']}, '37|91': {'11': ['01', '10']}, '37|92': {'11': ['01', '10']}, '37|93': {'11': ['01', '10']}, '37|94': {'11': ['01', '10']}, '37|95': {'11': ['01', '10']}, '37|96': {'11': ['01', '10']}, '37|97': {'11': ['01', '10']}, '37|98': {'11': ['01', '10']}, '37|99': {'11': ['01', '10']}, '38|99'

{'43|56': {'00': ['01', '11']}, '43|57': {'10': ['01']}, '44|57': {'00': ['11'], '10': ['01', '11']}, '44|58': {'10': ['01', '11']}, '44|59': {'10': ['01']}, '45|59': {'00': ['11'], '10': ['01', '11']}, '45|60': {'10': ['01']}, '46|60': {'00': ['11'], '10': ['01', '11']}, '46|61': {'10': ['01', '11']}, '46|62': {'10': ['01', '11']}, '46|63': {'10': ['01']}, '47|63': {'00': ['11'], '10': ['01', '11']}, '47|64': {'10': ['01', '11']}, '47|65': {'10': ['01', '11']}, '47|66': {'10': ['01']}, '48|66': {'00': ['11'], '10': ['01', '11']}, '48|67': {'10': ['01', '11']}, '48|68': {'10': ['01', '11']}, '48|69': {'10': ['01']}, '49|69': {'00': ['11'], '10': ['11', '01']}, '49|70': {'10': ['11', '01']}, '49|71': {'10': ['01', '11']}, '49|72': {'10': ['01']}, '50|72': {'00': ['11'], '10': ['01', '11']}, '50|73': {'10': ['01', '11']}, '50|74': {'10': ['01', '11']}, '50|75': {'10': ['01']}, '51|75': {'00': ['11'], '10': ['01', '11']}, '51|76': {'10': ['01', '11']}, '51|77': {'10': ['01', '11']}, '51|7

{'43|62': {'00': ['01', '11']}, '43|63': {'10': ['01']}, '44|63': {'00': ['01', '11'], '10': ['01']}, '44|64': {'10': ['01']}, '45|64': {'00': ['11'], '10': ['11', '01']}, '45|65': {'10': ['01', '11']}, '45|66': {'10': ['01']}, '46|66': {'00': ['11'], '10': ['01', '11']}, '46|67': {'10': ['01', '11']}, '46|68': {'10': ['01', '11']}, '46|69': {'10': ['01']}, '47|69': {'00': ['11'], '10': ['01', '11']}, '47|70': {'10': ['01', '11']}, '47|71': {'10': ['01', '11']}, '47|72': {'10': ['01']}, '48|72': {'00': ['11'], '10': ['01', '11']}, '48|73': {'10': ['01', '11']}, '48|74': {'10': ['01', '11']}, '48|75': {'10': ['01']}, '49|75': {'00': ['11'], '10': ['11', '01']}, '49|76': {'10': ['11', '01']}, '49|77': {'10': ['01', '11']}, '49|78': {'10': ['01']}, '50|78': {'00': ['11'], '10': ['01', '11']}, '50|79': {'10': ['01', '11']}, '50|80': {'10': ['01', '11']}, '50|81': {'10': ['01']}, '51|81': {'00': ['11'], '10': ['01', '11']}, '51|82': {'10': ['01', '11']}, '51|83': {'10': ['01', '11']}, '51|8

...

{'49|56': {'00': ['01', '11']}, '49|57': {'10': ['01', '11']}, '49|58': {'10': ['01', '11']}, '49|59': {'10': ['01']}, '50|59': {'00': ['11'], '10': ['01', '11']}, '50|60': {'10': ['01', '11']}, '50|61': {'10': ['01', '11']}, '50|62': {'10': ['01']}, '51|62': {'00': ['11'], '10': ['01', '11']}, '51|63': {'10': ['01', '11']}, '51|64': {'10': ['01', '11']}, '51|65': {'10': ['01']}, '52|65': {'00': ['11'], '10': ['01', '11']}, '52|66': {'10': ['01', '11']}, '52|67': {'10': ['01']}, '53|67': {'00': ['11'], '10': ['01', '11']}, '53|68': {'10': ['01', '11']}, '53|69': {'10': ['01']}, '54|69': {'00': ['01', '11'], '10': ['01']}, '54|70': {'10': ['01']}, '55|70': {'00': ['01', '11'], '10': ['01', '10'], '11': ['01']}, '55|71': {'10': ['01', '10'], '11': ['01']}, '55|72': {'11': []}, '56|71': {'00': ['10', '01'], '11': ['01']}, '56|72': {'11': ['01']}, '57|72': {'00': ['10'], '11': ['01', '10']}, '57|73': {'11': ['01']}, '58|73': {'00': ['10'], '11': ['10', '01']}, '58|74': {'11': ['10', '01']}

{'49|62': {'00': ['01', '11']}, '49|63': {'10': ['01', '11']}, '49|64': {'10': ['01', '11']}, '49|65': {'10': ['01']}, '50|65': {'00': ['11'], '10': ['01', '11']}, '50|66': {'10': ['01', '11']}, '50|67': {'10': ['01', '11']}, '50|68': {'10': ['01']}, '51|68': {'00': ['11'], '10': ['01', '11']}, '51|69': {'10': ['01', '11']}, '51|70': {'10': ['01', '11']}, '51|71': {'10': ['01']}, '52|71': {'00': ['11'], '10': ['01', '11']}, '52|72': {'10': ['01', '11']}, '52|73': {'10': ['01', '11']}, '52|74': {'10': ['01']}, '53|74': {'00': ['11'], '10': ['01', '11']}, '53|75': {'10': ['01', '11']}, '53|76': {'10': ['01']}, '54|76': {'00': ['01', '11'], '10': ['01']}, '54|77': {'10': ['01']}, '55|77': {'00': ['01'], '10': ['01', '10'], '11': []}, '56|77': {'00': ['01', '10'], '11': ['01']}, '56|78': {'11': ['01'], '00': []}, '57|78': {'00': ['10'], '11': ['01', '10']}, '57|79': {'11': ['01']}, '58|79': {'00': ['10'], '11': ['10', '01']}, '58|80': {'11': ['10', '01']}, '58|81': {'11': ['01', '10']}, '5

{'49|68': {'00': ['01', '11']}, '49|69': {'10': ['01', '11']}, '49|70': {'10': ['01', '11']}, '49|71': {'10': ['01']}, '50|71': {'00': ['11'], '10': ['01', '11']}, '50|72': {'10': ['01', '11']}, '50|73': {'10': ['01', '11']}, '50|74': {'10': ['01']}, '51|74': {'00': ['11'], '10': ['01', '11']}, '51|75': {'10': ['01', '11']}, '51|76': {'10': ['01', '11']}, '51|77': {'10': ['01']}, '52|77': {'00': ['11'], '10': ['01', '11']}, '52|78': {'10': ['01', '11']}, '52|79': {'10': ['01', '11']}, '52|80': {'10': ['01']}, '53|80': {'00': ['11'], '10': ['01', '11']}, '53|81': {'10': ['01']}, '54|81': {'00': ['01', '11'], '10': ['01']}, '54|82': {'10': ['01']}, '55|82': {'00': ['01', '11'], '10': ['01', '10'], '11': ['01']}, '55|83': {'10': ['01', '10'], '11': ['01']}, '55|84': {'11': []}, '56|83': {'00': ['01', '10'], '11': ['01']}, '56|84': {'11': ['01']}, '57|84': {'00': ['10'], '11': ['01', '10']}, '57|85': {'11': ['01']}, '58|85': {'00': ['10'], '11': ['10', '01']}, '58|86': {'11': ['10', '01']}

{'56|62': {'00': ['01', '10']}, '56|63': {'11': ['01']}, '57|63': {'00': ['10'], '11': ['01', '10']}, '57|64': {'11': ['01', '10']}, '57|65': {'11': ['01']}, '58|65': {'00': ['10'], '11': ['10', '01']}, '58|66': {'11': ['10', '01']}, '58|67': {'11': ['01', '10']}, '58|68': {'11': ['01']}, '59|68': {'00': ['10'], '11': ['10', '01']}, '59|69': {'11': ['10', '01']}, '59|70': {'11': ['01', '10']}, '59|71': {'11': ['01', '10']}, '59|72': {'11': ['01', '10']}, '59|73': {'11': ['01', '10']}, '59|74': {'11': ['01', '10']}, '59|75': {'11': ['01', '10']}, '59|76': {'11': ['01', '10']}, '59|77': {'11': ['01', '10']}, '59|78': {'11': ['01', '10']}, '59|79': {'11': ['01', '10']}, '59|80': {'11': ['01', '10']}, '59|81': {'11': ['01', '10']}, '59|82': {'11': ['01', '10']}, '59|83': {'11': ['01', '10']}, '59|84': {'11': ['01', '10']}, '59|85': {'11': ['01', '10']}, '59|86': {'11': ['01', '10']}, '59|87': {'11': ['01', '10']}, '59|88': {'11': ['01', '10']}, '59|89': {'11': ['01', '10']}, '59|90': {'11'

{'62|56': {'00': ['10']}, '62|57': {'11': ['10']}, '62|58': {'11': ['10']}, '62|59': {'11': ['10']}, '62|60': {'11': ['10']}, '62|61': {'11': ['10']}, '62|62': {'11': ['10']}, '62|63': {'11': ['10']}, '62|64': {'11': ['10']}, '62|65': {'11': ['10']}, '62|66': {'11': ['10']}, '62|67': {'11': ['10']}, '62|68': {'11': ['10']}, '62|69': {'11': ['10']}, '62|70': {'11': ['10']}, '62|71': {'11': ['10']}, '62|72': {'11': ['10']}, '62|73': {'11': ['10']}, '62|74': {'11': ['10']}, '62|75': {'11': ['10']}, '62|76': {'11': ['10']}, '62|77': {'11': ['10']}, '62|78': {'11': ['10']}, '62|79': {'11': ['10']}, '62|80': {'11': ['10']}, '62|81': {'11': ['10']}, '62|82': {'11': ['10']}, '62|83': {'11': ['10']}, '62|84': {'11': ['10']}, '62|85': {'11': ['10']}, '62|86': {'11': ['10']}, '62|87': {'11': ['10']}, '62|88': {'11': ['10']}, '62|89': {'11': ['10']}, '62|90': {'11': ['10']}, '62|91': {'11': ['10']}, '62|92': {'11': ['10']}, '62|93': {'11': ['10']}, '62|94': {'11': ['10']}, '62|95': {'11': ['10']},

In [10]:
#Experiments with one init state...

#K defined in the first cell
#visited_vanderpol = find_all_reachable_from(  K, [60,50], [0,0], [0,K], vanderpol_f, vanderpol_f_auton )
#visited_vanderpol = find_all_reachable_from(  K, [62,62], [1,1], [0,K], vanderpol_f, vanderpol_f_auton )
#visited_vanderpol = find_all_reachable_from(  K, [43,37], [0,1], [0,K], vanderpol_f, vanderpol_f_auton )
##visited_vanderpol = find_all_reachable_from(  K, [43,37], [1,0], [0,K], vanderpol_f, vanderpol_f_auton )
#visited_vanderpol = find_all_reachable_from(  K, [62,49], [0,1], [0,K], vanderpol_f, vanderpol_f_auton )
##visited_vanderpol = find_all_reachable_from(  K, [43,25], [0,1], [0,K], vanderpol_f, vanderpol_f_auton )

In [ ]:
#Try with longer maxT
maxT = 5.0

#Find forward reachable sets for rectangles (entry [] the whole rectangle) in init
#write out the results - name them with identifiers "_x_y"
initial_rectangles = []
identifiers = []
for x in [-3, -2, -1, 0, 1, 2, 3]:
    for y in [-3, -2, -1, 0, 1, 2, 3]:
        initial_rectangles.append( find_rectangle( [ x, y ] ) )
        identifiers.append( "_" + str( x ) + "_" + str( y ) )

for r,i in zip( initial_rectangles, identifiers ):
    visited_vdp = find_all_reachable_from(  K, r, [0,0], [], vanderpol_f, vanderpol_f_auton, i+"T5" )
    #print( visited_vdp )
